## Librerias que utilizaremos 

In [13]:
from pymongo import MongoClient
from pymongo import GEOSPHERE

## Conectamos con Mongo DB

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [3]:
client.list_database_names()

['Ironhack', 'Taller_Geo', 'admin', 'config', 'local']

## Obtenemos la Base de datos con la que trabajaremos

In [4]:
db = client.get_database("Taller_Geo")

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Taller_Geo')

In [6]:
db.list_collection_names()

['Bucarest', 'Dublin', 'Madrid']

In [7]:
B = db.get_collection("Bucarest")

In [8]:
B

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Taller_Geo'), 'Bucarest')

In [9]:
M = db.get_collection("Madrid")

In [10]:
M

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Taller_Geo'), 'Madrid')

In [11]:
D = db.get_collection("Dublin")

In [12]:
D

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Taller_Geo'), 'Dublin')

## Creamos los índices para cada colección

In [14]:
db.Bucarest.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [15]:
db.Madrid.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [16]:
db.Dublin.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [20]:
#db.Dublin.find_one() #comprobamos que las 3 colecciones nos brindan información correctamente

## Utilizamos Near para ver la cercanía de los datos a nuestro punto central

In [107]:
#Estos son mis puntos centrales:
madrid_coord = [40.42955,-3.6793]
dublin_coord = [53.34919,-6.2606] 
bucarest_coord = [44.42724,26.09208]

In [108]:
#Esta es la distancia máxima que quiero 
metros = 5000

In [109]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [110]:
coord_tp_M = type_point(madrid_coord)
coord_tp_D = type_point(dublin_coord)
coord_tp_B = type_point(bucarest_coord)

In [111]:
coord_tp_M 

{'type': 'Point', 'coordinates': [40.42955, -3.6793]}

In [112]:
query_M = {"location": {"$near": {"$geometry": coord_tp_M, "$maxDistance": metros}}}

In [113]:
query_M

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [40.42955, -3.6793]},
   '$maxDistance': 5000}}}

In [114]:
resultado_M = list(M.find(query_M))
#resultado_M

In [115]:
query_D = {"location": {"$near": {"$geometry": coord_tp_D, "$maxDistance": metros}}}

In [116]:
query_D

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [53.34919, -6.2606]},
   '$maxDistance': 5000}}}

In [117]:
resultado_D = list(D.find(query_D))
#resultado_D

In [118]:
coord_tp_B

{'type': 'Point', 'coordinates': [44.42724, 26.09208]}

In [119]:
query_B = {"location": {"$near": {"$geometry": coord_tp_B, "$maxDistance": metros}}}

In [121]:
resultado_B = list(B.find(query_B))
#resultado_B

In [123]:
resultado_B[0]

{'_id': ObjectId('618fac476bd838c298f8a588'),
 'nombre': 'bella dog',
 'latitud': 44.429934133909114,
 'longitud': 26.09415232523072,
 'location': {'type': 'Point',
  'coordinates': [44.429934133909114, 26.09415232523072]}}

In [ ]:
total = [] # lista para resultado final
for libreria in libres_cerca: #iteramos por la lista entera de librerías de antes 
    point = type_point([libreria["longitud"],libreria["latitud"]]) #preparamos tipo point de cada una 
    query = {"geometry": {"$geoIntersects": {"$geometry": point}}} #hacemos la query para buscar el barrio 
    resultado = dis.find_one(query) # hacemos la query 
    libre = {libreria["nombre"]:resultado["properties"]["nombre"]} # nombre de la librería y nombre del distrito de la query
    total.append(libre)